In [1]:
import os
import sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import rsi_below_entries, plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('../30min.csv', index_col='time')

entries = rsi_below_entries(
    timeperiods=[15],
    below_ranges=[40],
    prices=prices.close
)


In [2]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    gains_pct_filter=40,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [3]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.iloc[3009:3013]

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
3009,3009,2,6946,486.66,"53,211.50","52,662.53",NaN,LongEntry,NaN,"2,003.88",NaN,"50,556.02","65,744.16"
3010,3010,2,6956,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,"2,003.88",NaN,"52,672.46","65,744.16"


In [4]:
df_or.iloc[1145:1155]

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
1145,1145,0,6951,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"52,675.43","56,075.21"
1146,1146,0,6952,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"52,904.11","56,075.21"
1147,1147,0,6953,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,229.82","56,075.21"
1148,1148,0,6955,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,619.89","56,075.21"
1149,1149,0,6956,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,743.04,NaN,"53,758.49","56,075.21"
1150,1150,0,6958,"1,402.76","53,758.49","52,662.53",1.70,LongTSL,27.49,770.53,NaN,"53,758.49","56,075.21"
1151,1151,1,31,320.70,"10,850.00","10,850.00",NaN,LongEntry,NaN,"1,000.00",NaN,"10,524.50","12,555.67"
1152,1152,1,32,320.70,"10,725.00","10,787.14",NaN,LongEntry,NaN,"1,000.00",NaN,"10,463.52","12,482.92"
1153,1153,1,33,320.70,"10,577.50","10,716.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,394.85","12,401.00"
1154,1154,1,34,320.70,"10,523.00","10,667.34",NaN,LongEntry,NaN,"1,000.00",NaN,"10,347.32","12,344.30"


In [5]:
df_or.tail()

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
3006,3006,2,6941,486.66,"51,319.50","51,319.50",NaN,LongEntry,NaN,"2,003.88",NaN,"49,266.72","64,067.52"
3007,3007,2,6942,486.66,"53,278.00","52,280.41",NaN,LongEntry,NaN,"2,003.88",NaN,"50,189.20","65,267.13"
3008,3008,2,6943,486.66,"52,890.00","52,482.04",NaN,LongEntry,NaN,"2,003.88",NaN,"50,382.76","65,518.85"
3009,3009,2,6946,486.66,"53,211.50","52,662.53",NaN,LongEntry,NaN,"2,003.88",NaN,"50,556.02","65,744.16"
3010,3010,2,6956,NaN,NaN,"52,662.53",NaN,MovedTSL,NaN,"2,003.88",NaN,"52,672.46","65,744.16"


In [6]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,195.00,118.00,77.00
gains_pct,-22.95,16.51,100.39
win_rate,37.95,45.76,51.95
to_the_upside,0.67,0.54,0.29
total_pnl,-229.47,165.12,"1,003.88"
ending_eq,770.53,"1,165.12","2,003.88"
settings_id,0,1,2
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [7]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [8]:
clear_cache()